<a href="https://colab.research.google.com/github/tyslas/CS5265-tyslas-nfl-spread-line-outcomes/blob/main/NFLSpreadAndLineOutcomes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Project to predict future NFL Spread & Line outcomes

## Author: Tito Yslas

### Background
I enjoy watching the NFL and playing fantasy football. I also like to place bets on games using apps like FanDuel. The purpose of this project is to increase my understanding of the NFL betting market and possibly create a machine learning model to give myself an edge next season.

### Project Description
I found a dataset from Kaggle titled [NFL scores and betting data](https://www.kaggle.com/datasets/tobycrabtree/nfl-scores-and-betting-data?resource=download). This dataset has over 13,000 samples and 17 features. The goal is to use this dataset to train a model that will predict the winner of a given game.

### Performance Metric
For my performance metric, I will aim for the model to have at least 70% accuracy.

## Import Libraries

In [3]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

## Load Data

### Data Dictionary
- schedule_date: Date that the game took place. This column is a date in the format MM/DD/YYYY
- schedule_season: Year of the season began. NFL seasons start in the fall and end before the spring of the following year. So the 2023 season will refer to the years 2023-2024. This column is a number in the format YYYY
- schedule_week: Week of the NFL season. This column is either a number during regular season weeks or a string in playoff weeks. For the purposes of this project the playoff weeks will be converted to numbers and the schedule_playoff column will be used to determine whether the week is a playoff game or not
- schedule_playoff: This column is a boolean. FALSE is regular season and TRUE is playoffs
- team_home: Name of the home team. This column is a string
- score_home: Points scored by the home team. This column is a number
- score_away: Points scored by the away team. This column is a number
- team_away: Name of the away team. This column is a string
- team_favorite_id: Acronym of the team that was determined most likely to win by the betting market. It is either two or three letters. For the purposes of this project this column will be changed to be either the team_home or team_away name
- team_home_favorite: this will represent the encoded team_favorite_id - if team_home is favored this column will be marked as a 1 - if it's a zero then we know that team_away is favored
- spread_favorite: The number of points that the favored team needs to win by for a bet placed on the spread of the favorite to win. This column will either be a negative number or zero
- over_under_line: The number of points that both teams combined need to score for a bet placed on the 'line' to win. This column is a positive number
- stadium: Name of the venue that the game is played
- stadium_neutral: This column is a boolean. FALSE is not a neutral venue and TRUE is a neutral venue - for the purposes of this project this column will be one hot encoded with a neutral venue being marked as a 1 and non-neutral marked as a 0
- weather_temperature: The temperature in Fahrenheit at the venue where the game is played. This column is a number
- weather_wind_mph: The speed of wind in miles per hour. This column is a number
- weather_humidity: The measurement of water vapor in the air during the game measured as a percentage. This column is a number
- weather_detail: Other information about the weather conditions - if the venue is indoor or the venue has a retractable roof. This column is a string


In [7]:
from google.colab import drive
drive.mount('/content/drive/')
# change directory to relevant files
%cd /content/drive/MyDrive/vandy/nfl
# print all file names in directory
for file in os.listdir():
  print(file)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/vandy/nfl
data_dictionary.csv
spreadspoke_scores.csv


## Read NFL Scores csv

In [9]:
scores = pd.read_csv('spreadspoke_scores.csv')
scores

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,9/2/66,1966,1,False,Miami Dolphins,14,23,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71.0,NaN
1,9/3/66,1966,1,False,Houston Oilers,45,7,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70.0,NaN
2,9/4/66,1966,1,False,San Diego Chargers,27,7,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82.0,NaN
3,9/9/66,1966,2,False,Miami Dolphins,14,19,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78.0,NaN
4,9/10/66,1966,1,False,Green Bay Packers,24,3,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13511,1/22/23,2022,Division,True,Buffalo Bills,10,27,Cincinnati Bengals,BUF,-6.0,48.5,Highmark Stadium,False,32.0,4.0,100.0,snow
13512,1/22/23,2022,Division,True,San Francisco 49ers,19,12,Dallas Cowboys,SF,-3.5,46.5,Levi's Stadium,False,55.0,19.0,47.0,NaN
13513,1/29/23,2022,Conference,True,Kansas City Chiefs,23,20,Cincinnati Bengals,KC,-1.5,48,GEHA Field at Arrowhead Stadium,False,22.0,13.0,55.0,NaN
13514,1/29/23,2022,Conference,True,Philadelphia Eagles,31,7,San Francisco 49ers,PHI,-2.5,45.5,Lincoln Financial Field,False,52.0,14.0,48.0,rain


In [10]:
scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13516 entries, 0 to 13515
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   schedule_date        13516 non-null  object 
 1   schedule_season      13516 non-null  int64  
 2   schedule_week        13516 non-null  object 
 3   schedule_playoff     13516 non-null  bool   
 4   team_home            13516 non-null  object 
 5   score_home           13516 non-null  int64  
 6   score_away           13516 non-null  int64  
 7   team_away            13516 non-null  object 
 8   team_favorite_id     11037 non-null  object 
 9   spread_favorite      11037 non-null  float64
 10  over_under_line      11027 non-null  object 
 11  stadium              13516 non-null  object 
 12  stadium_neutral      13516 non-null  bool   
 13  weather_temperature  12309 non-null  float64
 14  weather_wind_mph     12293 non-null  float64
 15  weather_humidity     8468 non-null  